In [1]:
import re
from collections import defaultdict
from collections import Counter
import numpy as np
import copy

with open('day_20.txt', 'r') as f:
    tiles = f.read().split('\n\n')
    
tiles = [tile.splitlines() for tile in tiles]

# Part 1

In [2]:
edge_dict = defaultdict(list)
tile_dict = dict()

for t in tiles:
    
    tile_id = int(re.search(r'([0-9]+)', t[0]).group(1))
    tile = t[1:]
    tile_dict[tile_id] = tile
    
    # get four borders
    up = tile[0]
    down = tile[-1]
    left = ''.join([x[0] for x in tile])
    right = ''.join([x[-1] for x in tile])
    
    # get a tuple of the border and the reversal of the border
    # add to dictionary with the tile id as value
    edge_dict[frozenset((up, up[::-1]))].append(tile_id)
    edge_dict[frozenset((down, down[::-1]))].append(tile_id)
    edge_dict[frozenset((left, left[::-1]))].append(tile_id)
    edge_dict[frozenset((right, right[::-1]))].append(tile_id)
    

In [3]:
# get all edges that only exist in one tile (borders)
lone_edges = [l[0] for l in edge_dict.values() if len(l) == 1]
# get tile IDs that show up twice in the list of borders (corners)
corners = [key for key, val in Counter(lone_edges).items() if val == 2]
np.prod([float(c) for c in corners])

47213728755493.0

# Part 2

## Get the image

In [4]:
def rotate_tile(tile):
    """Rotate a tile clockwise."""
    return([''.join(x) for x in list(zip(*reversed(tile)))])

In [5]:
def flip_tile(tile):
    """Mirror a tile."""
    return([x[::-1] for x in tile])

In [6]:
def orient_tile(left_tile, right_tile):
    """Flip and rotate tile 2 until its left edge aligns with right edge of tile 1."""  
    
    right = ''.join([x[-1] for x in left_tile])
    
    for i in range(8):
        
        right_tile = rotate_tile(right_tile)
        left = ''.join([x[0] for x in right_tile])
        if right == left:
            return(right_tile)
        
        right_tile = flip_tile(right_tile)
        left = ''.join([x[0] for x in right_tile])
        if right == left:
            return(right_tile)

        if i % 2 == 0:
            right_tile = rotate_tile(right_tile)

In [7]:
def orient_tile_updown(up_tile, down_tile):
    """Flip and rotate tile 2 until its upper edge aligns with bottom edge of tile 1."""  
      
    down = up_tile[-1]
    
    for i in range(8):
        
        down_tile = rotate_tile(down_tile)
        up = down_tile[0]
        if up == down:
            return(down_tile)
        
        down_tile = flip_tile(down_tile)
        up = down_tile[0]
        if up == down:
            return(down_tile)
        
        if i % 2 == 0:
            down_tile = rotate_tile(down_tile)

In [8]:
# get a starting corner tile from part 1 to be our "upper left" corner and orient it properly
# this can be multiple tiles because of flipping but we can always flip the final image
starting_corner_id = corners[0]
starting_corner = tile_dict[corners[0]]

while True:
    
    right = ''.join([x[-1] for x in starting_corner])
    down = starting_corner[-1]
    
    # check if the right edge and bottom edge are shared
    if len(edge_dict[frozenset((right, right[::-1]))]) == 2 and len(edge_dict[frozenset((down, down[::-1]))]) == 2:
        break
            
    # if not, rotate  the tile
    starting_corner = rotate_tile(starting_corner)
    
tile_dict[starting_corner_id] = starting_corner

In [9]:
tile_order = [starting_corner_id]

In [10]:
while len(tile_order) < 144:
    
    if len(tile_order) % 12 == 0:
        # grab the beginning of the previous row
        previous_tile_id = tile_order[len(tile_order)-12]
        previous_tile = tile_dict[previous_tile_id]
        right = previous_tile[-1]
        
    else:
        # grab the previous tile in the row
        previous_tile_id = tile_order[-1]
        previous_tile = tile_dict[previous_tile_id]
        right = ''.join([x[-1] for x in previous_tile])
    
    # get the tile that borders the right edge
    right_neighbor_id = copy.deepcopy(edge_dict[frozenset((right, right[::-1]))])
    right_neighbor_id.remove(previous_tile_id)
    right_neighbor_id = right_neighbor_id[0]
    right_neighbor = copy.deepcopy(tile_dict[right_neighbor_id])
    
    # orient the tile
    if len(tile_order) % 12 == 0:
        right_neighbor = orient_tile_updown(previous_tile, right_neighbor)
    else:
        right_neighbor = orient_tile(previous_tile, right_neighbor)
        
    # add it to the list and properly orient the tile in the dict
    tile_dict[right_neighbor_id] = right_neighbor
    tile_order.append(right_neighbor_id)

In [11]:
def remove_border(tile):
    return([x[1:len(x)-1] for x in tile[1:len(tile)-1]])

In [12]:
# remove the borders and concat into a single "image"
start = [remove_border(tile_dict[tile_order[0]])]
for i in range(1, 144):
    
    new = remove_border(tile_dict[tile_order[i]])
    
    if i % 12 == 0:
        start.append(new)
    else:
        start[-1] = [''.join(x) for x in list(zip(start[-1], new))]

In [13]:
pic = [item for sublist in start for item in sublist]

## Find the serpents

In [14]:
def get_serpents(p):
    """Brute force serpent identification because I'm very very tired."""
    serpent_tiles = []
    for i in range(8):
        for y in range(1, len(p)-1):
            for x in range(len(p[y])-19):
                if p[y][x] != '#':
                    continue
                if p[y+1][x+1] != '#':
                    continue
                if p[y+1][x+4] != '#':
                    continue
                if p[y][x+5] != '#':
                    continue
                if p[y][x+6] != '#':
                    continue
                if p[y+1][x+7] != '#':
                    continue
                if p[y+1][x+10] != '#':
                    continue
                if p[y][x+11] != '#':
                    continue
                if p[y][x+12] != '#':
                    continue
                if p[y+1][x+13] != '#':
                    continue
                if p[y+1][x+16] != '#':
                    continue
                if p[y][x+17] != '#':
                    continue
                if p[y-1][x+18] != '#':
                    continue
                if p[y][x+18] != '#':
                    continue
                if p[y][x+19] != '#':
                    continue

                serpent_tiles.append((x, y))
                serpent_tiles.append((x+1, y+1))
                serpent_tiles.append((x+4, y+1))
                serpent_tiles.append((x+5, y))
                serpent_tiles.append((x+6, y))
                serpent_tiles.append((x+7, y+1))
                serpent_tiles.append((x+10, y+1))
                serpent_tiles.append((x+11, y))
                serpent_tiles.append((x+12, y))
                serpent_tiles.append((x+13, y+1))
                serpent_tiles.append((x+16, y+1))
                serpent_tiles.append((x+17, y))
                serpent_tiles.append((x+18, y-1))
                serpent_tiles.append((x+18, y))
                serpent_tiles.append((x+19, y))
                
        if len(serpent_tiles) > 0:
            return(serpent_tiles, p)
        
        p = rotate_tile(p)


In [15]:
serpents, rotated_pic = get_serpents(pic)

In [16]:
sum([x.count('#') for x in pic]) - len(serpents)

1599